# Import des outils / jeu de données

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from lifelines import KaplanMeierFitter
import matplotlib.pyplot as plt

import os

In [2]:
os.getcwd()

'/home/ab2/Documents/GitHub/ab2/analyse-survie/src'

In [3]:
np.random.seed(0)
sns.set_theme()

In [ ]:
df = pd.read_csv(
    "../data/kickstarter_1.csv",
    parse_dates=True,
)

# Kaplan-Meier

In [ ]:
sns.histplot(df["day_succ"])

In [ ]:
kmf1 = KaplanMeierFitter()

event_times = df["day_succ"]
event_observed = df["Status"]
kmf1.fit(event_times, event_observed)

In [ ]:
plt.figure(figsize=(12, 6))
kmf1.plot()
plt.title(
    "Probabilité que le projet n'ait pas encore été financé - données censurées incluses"
)
plt.xlabel("Nombre de jours écoulés depuis le lancement du projet")
plt.ylabel("Probabilité")

In [ ]:
kmf2 = KaplanMeierFitter()

event_times = df["day_succ"][df["Status"] == 1]
event_observed = df["Status"][df["Status"] == 1]

kmf2.fit(event_times, event_observed)

In [ ]:
plt.figure(figsize=(12, 6))
kmf2.plot()
plt.title(
    "Probabilité que le projet n'ait pas encore été financé - données censurées exclues"
)
plt.xlabel("Nombre de jours écoulés depuis le lancement du projet")
plt.ylabel("Probabilité")

In [ ]:
kmf1 = KaplanMeierFitter()

event_times = df["day_succ"]
event_observed = df["Status"]
kmf1.fit(event_times, event_observed)
plt.figure(figsize=(12, 6))
kmf1.plot_survival_function(label="Avec censure")
kmf2 = KaplanMeierFitter()

event_times = df["day_succ"][df["Status"] == 1]
event_observed = df["Status"][df["Status"] == 1]

kmf2.fit(event_times, event_observed)

kmf2.plot_survival_function(label="Sans censure")
plt.title(
    "Probabilité que le projet n'ait pas encore été financé\n- données censurées vs non censurées\n(estimation de Kaplan-Meier)"
)
plt.xlabel("Nombre de jours écoulés depuis le lancement du projet")
plt.ylabel("Probabilité")

# Kaplan Meier par classe

In [ ]:
df_video = df[df["has_video"] == 1].copy()
df_no_video = df[df["has_video"] == 0].copy()

In [ ]:
plt.figure(figsize=(12, 6))
kmf = KaplanMeierFitter()

event_times = df["day_succ"]
event_observed = df["Status"]
kmf.fit(event_times, event_observed)
kmf.plot_survival_function(label="population complète")


event_times = df_video["day_succ"]
event_observed = df_video["Status"]
kmf.fit(event_times, event_observed)

kmf.plot_survival_function(label="avec vidéo")


event_times = df_no_video["day_succ"]
event_observed = df_no_video["Status"]
kmf.fit(event_times, event_observed)

kmf.plot_survival_function(label="sans vidéo")

plt.title(
    """Probabilité que le projet n'ait pas encore été financé\nen fonction du critère "possède une vidéo"\n(estimation de Kaplan-Meier)"""
)


plt.xlabel("Nombre de jours écoulés depuis le lancement du projet")
plt.ylabel("Probabilité")